In [5]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain.chat_models import ChatOpenAI

In [14]:
from dotenv import load_dotenv
load_dotenv()


True

In [15]:
key=os.getenv("OPENAI_API_KEY")

In [17]:
print(key)

sk-proj-nT6rtF7rS1F8D4ZXJj64T3BlbkFJraDoJXZYPJTgqyd2CGzv


In [18]:
llm = ChatOpenAI(openai_api_key=key,temperature=0.7)

In [19]:
prompt ="who was the first captain of Afghan cricket team"

c:\Users\fayab\MCQGEN\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


TypeError: Got unknown type w